In [1]:
import math
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader
from torchtext import data
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
from tqdm import tqdm
import time
import datetime
import itertools
import pickle
import sys
import copy
import gc
import os

In [2]:
# pickle書き込み
def write_pickle(filepath, data):
    start_time = time.time()
    print(f'writing pickle to "{filepath}" ...')    
    
    with open(filepath, 'wb') as p:
        pickle.dump(data,p)
    
    print(f'end of writeing {time.time()-start_time:6.2f} s')
    
    del start_time
    gc.collect()

In [3]:
# pickle書き込み
# ログなしVer
def write_pickle_quickly(filepath, data):
    with open(filepath, 'wb') as p:
        pickle.dump(data,p)

In [4]:
# pickle読み出し
def read_pickle(filepath):
    start_time = time.time()
    print(f'reading pickle from "{filepath}" ...')
    
    with open(filepath, 'rb') as p:
        data = pickle.load(p)
    
    print(f'end of reading {time.time()-start_time:6.2f} s')
    
    del start_time
    gc.collect()
    
    return data

In [5]:
# pickle読み出し
# ログなしVer
def read_pickle_quickly(filepath):
    with open(filepath, 'rb') as p:
        data = pickle.load(p)
    return data

In [6]:
# vacab作成
# テキストを単語で分割

v_start = time.time()
print("Reading...")
vocab = read_pickle('../../external_drive/pickle/vocab.pickle')
print('Finish!!')
print(f'{time.time() - v_start:5.2f} s')
del v_start
gc.collect()

'''
v_start = time.time()
tokenizer = get_tokenizer('basic_english')

# data field定義
TEXT  = data.Field(sequential=True,
                     lower=True,
                     batch_first=True, 
                     tokenize=tokenizer,
                     init_token='<cls>')

# CSVファイルを読み込み、TabularDatasetオブジェクトの作成
print("Reading...")
vocab_data = data.TabularDataset(path ='tweet-transformer/1d/2021-17_t.csv',
                                       format='csv',
                                       skip_header = True,
                                       fields=[('tweet', TEXT)])

# 単語辞書の作成
print("Creating vocab...")
TEXT.build_vocab(vocab_data, min_freq=3)
vocab = TEXT.vocab
print(f'{len(vocab)=}')

print('Finish!!')
print(f'{time.time() - v_start:5.2f} s')

# メモリ開放
del v_start, vocab_data, tokenizer, TEXT
gc.collect()
'''

Reading...
reading pickle from "../../external_drive/pickle/vocab.pickle" ...
end of reading   1.86 s
Finish!!
 1.94 s


'\nv_start = time.time()\ntokenizer = get_tokenizer(\'basic_english\')\n\n# data field定義\nTEXT  = data.Field(sequential=True,\n                     lower=True,\n                     batch_first=True, \n                     tokenize=tokenizer,\n                     init_token=\'<cls>\')\n\n# CSVファイルを読み込み、TabularDatasetオブジェクトの作成\nprint("Reading...")\nvocab_data = data.TabularDataset(path =\'tweet-transformer/1d/2021-17_t.csv\',\n                                       format=\'csv\',\n                                       skip_header = True,\n                                       fields=[(\'tweet\', TEXT)])\n\n# 単語辞書の作成\nprint("Creating vocab...")\nTEXT.build_vocab(vocab_data, min_freq=3)\nvocab = TEXT.vocab\nprint(f\'{len(vocab)=}\')\n\nprint(\'Finish!!\')\nprint(f\'{time.time() - v_start:5.2f} s\')\n\n# メモリ開放\ndel v_start, vocab_data, tokenizer, TEXT\ngc.collect()\n'

In [7]:
# Dataset1の定義
# args　：tdf['ids'], tdf['mask']
# return：dataset{ids,mask}

class CreateDataset1(Dataset):
    def __init__(self, x, y):
        self.x = x # tdf['ids']
        self.y = y # tdf['mask']
        
    # len(Dataset)で返す値を指定
    def __len__(self):
        return len(self.x)

    # Dataset[index]で返す値を指定
    def __getitem__(self, index):
        ids  = self.x[index]
        mask = self.y[index]

        return {'ids'   : ids,
                'mask'  : mask}

In [8]:
# Dataset3の定義
# args　：tensor of section, tensor of price, tensor of trend(n+1)
# return：dataset3{section, src, target}
class CreateDataset3(Dataset):
    def __init__(self, x, y, z):
        self.x = x # tensor of section
        self.y = y # tensor of price
        self.z = z # tensor of trend(n+1)
        
    # len(Dataset)で返す値を指定
    def __len__(self):
        return len(self.x)

    # Dataset[index]で返す値を指定
    def __getitem__(self, index):
        section = self.x[index]
        src     = self.y[index]
        target  = self.z[index]

        return {'section': section,
                'src'    : src,
                'target' : target}

In [9]:
# tensor_df()内の関数
max_len = 128
tokenizer = get_tokenizer('basic_english')
def tokenize(text):
    return tokenizer(text)

def text_to_ids(tokenized_text):
    ids  = torch.tensor([vocab[word] for word in tokenized_text], dtype=torch.long).unsqueeze(0) # [1,seq_len]
    ids  = F.pad(ids, (0 ,max_len-len(tokenized_text)), "constant", 0) # [1,max_len]
    return ids

def ids_to_mask(ids):
    mask = (ids==0)
    return mask

In [10]:
# return df[section.ids,mask]
def tensor_df(df):
    start_time = time.time()
    
    print('1.text to ids...')
    df['tweet(n)'] = df['tweet(n)'].apply(tokenize)
    df['ids']      = df['tweet(n)'].apply(text_to_ids)
    print('2.ids to mask...')
    df['mask']     = df['ids'].apply(ids_to_mask)
    df = df.drop(columns=['tweet(n)'])
    
    print(f'end of df to tensor {time.time()-start_time:6.2f} s')
    
    return df

In [11]:
# return : df[section,ids(n),mask(n)]
# sectionごとにtensor連結
train_num_section_list = {'1d':181,'12h':363,'4h':1090,'1h':4361,'30m':8722,'15m':17444,'5m':52333}
test_num_section_list  = {'1d':31, '12h':61, '4h':182, '1h':727, '30m':1454,'15m':2908, '5m':8723}

def separate_section(df):
    num_section = train_num_section_list[timespan] + test_num_section_list[timespan]
    section_list = []
    ids_list  = []
    mask_list = []
    for i in range (0,num_section):
        #pandasのSectionkを抽出　idsとmaskをひとまとまりのtensorに
        section_list.append(i)
        df_ids = df[df['section'] == i]['ids']
        l = df_ids.values.tolist()
        x = torch.cat(l, dim=0)
        ids_list.append(x)
        df_mask = df[df['section'] == i]['mask']
        l = df_mask.values.tolist()
        x = torch.cat(l, dim=0)
        mask_list.append(x)
    
    df = pd.DataFrame(list(zip(section_list, ids_list, mask_list)), columns = ['section','ids','mask'])
    
    return df        

In [12]:
# Datasetの作成 (ツイート)
# 1. df to tensor
# 2. separate_section
def data_process1(timespan):
    print('-'*5 + 'Create dataset_tlist start!!' + '-'*5)

    print('Reading...')
    dfs = pd.read_csv(f'tweet-transformer/{timespan}/2021-17_s.csv')
    df = pd.read_csv(f'tweet-transformer/{timespan}/2021-17_t.csv')
    df = df.dropna(how='any')
    df = df.reset_index(drop=True)
    
    print('df to tensor...')
    df = tensor_df(df)
    
    print('Separating Section...')
    df = separate_section(df)
        
    train_tdf, test_tdf = train_test_split(df, test_size = 1/7, shuffle=False)
    
    print('Finish!!')
    print(f'{len(train_tdf)=}')
    print(f'{len(test_tdf)=}')
    
    return train_tdf, test_tdf

In [13]:
# Datasetの作成 (価格)
# 1.csv -> 3 tensor
# 2.CreateDataset3
def data_process2(timespan,n):
    print('-'*5 + 'Create dataset_plist start!!' + '-'*5)

    print('Reading...')
    df  = pd.read_csv(f'tweet-transformer/{timespan}/2021-17_b.csv')
    dfs = pd.read_csv(f'tweet-transformer/{timespan}/2021-17_s.csv')

    # 説明変数、目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)
    df['end_price(n)'] = df['open_price(n)'].shift(-1)
    if n >= 2:
        for i in range(1,n):
            df[f'trend(n-{i})'] = df['trend(n)'].shift(i)
            df[f'end_price(n-{i})'] = df['end_price(n)'].shift(i)
    df = df.drop(columns=['open_price(n)'])
    df = df.dropna(how='any')
    df = df.reset_index(drop=True)

    # マージして欠損値を含む行を処理
    df = pd.merge(dfs, df, on="section", how = 'left')
    df = df.reset_index(drop=True)
    df['bool'] = df.isnull().any(axis=1)
    for i in range(0, len(df)):
        if  df['bool'][i] == True:
            df['section'][i] = -1
    df = df.fillna(-1)
    df = df.drop(columns=['bool'])
    print(f'{len(df)=}')

    train_df, test_df = train_test_split(df, test_size = 1/7, shuffle=False)

    # 3つのtensorを作成
    print('Creating Dataset3...')
    section = torch.tensor(train_df['section'].values)
    target  = torch.tensor(train_df['trend(n+1)'].values)
    price   = torch.tensor(train_df.drop(columns=['trend(n+1)','section']).values)
    train_plist = CreateDataset3(section, price, target)

    section = torch.tensor(test_df['section'].values)
    target  = torch.tensor(test_df['trend(n+1)'].values)
    price   = torch.tensor(test_df.drop(columns=['trend(n+1)','section']).values)
    test_plist = CreateDataset3(section, price, target)


    print('Finish!!')
    print(f'{len(train_plist)=}')
    print(f'{len(test_plist)=}')
    
    del df,dfs,section,price,target,train_df,test_df
    gc.collect()
    
    return train_plist, test_plist

In [ ]:
# データセット作成 & csvに保存
# ツイート
tlist = ['1d','12h','4h','1h','30m','15m','5m']
train_section_list     = {'1d':181,'12h':363,'4h':1090,'1h':4361,'30m':8722,'15m':17444,'5m':52333}
test_section_list      = {'1d':31, '12h':61, '4h':182, '1h':727, '30m':1454,'15m':2908, '5m':8723}
tlist = ['1h']

for timespan in tlist:
    print('-'*50 + f'{timespan=}' + '-'*50)
    train_tdf, test_tdf = data_process1(timespan)
    print(f'{sys.getsizeof(train_tdf)=}')
    print(f'{sys.getsizeof(test_tdf)=}')
    train_tdf.to_csv(f'../../external_drive/pandas/{timespan}/train_tdf.csv', index=False)
    test_tdf.to_csv(f'../../external_drive/pandas/{timespan}/test_tdf.csv', index=False) 
    write_pickle(f'../../external_drive/pickle/{timespan}/train_tdf.pickle',train_tdf)
    write_pickle(f'../../external_drive/pickle/{timespan}/test_tdf.pickle',test_tdf)

In [ ]:
# データセット作成 & pickleに保存
# 価格
'''
nlist = [1,2,3,4,5,6,7,8,9,10]
tlist = ['1d','12h','4h','1h','30m','15m','5m']

for timespan, n in itertools.product(tlist, nlist):   

    print('-'*50 + f'{timespan=} / {n=}' + '-'*50)
    train_plist,test_plist = data_process2(timespan,n)
    write_pickle(f'../../external_drive/pickle/{timespan}/train_plist_{n}.pickle',train_plist)
    write_pickle(f'../../external_drive/pickle/{timespan}/test_plist_{n}.pickle',test_plist)
    
    del train_plist, test_plist
    gc.collect()


In [15]:
# parametator for Net
ntokens = len(vocab)  # size of vocabulary
d_model = 512   # embedding dimension
nhead   = 8     # number of heads in nn.MultiheadAttention
d_hid   = 2048  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6     # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
dropout = 0.2   # dropout probability
lstm_input_dim  = 16
lstm_hidden_dim = 1

In [16]:
# Transformer-LSTMモデルの概要
class Transformer(nn.Module):

    def __init__(self,
                 ntoken: int,
                 d_model: int,
                 nhead: int,
                 d_hid: int,
                 nlayers: int,
                 dropout: float = 0.5):

        super().__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model
        self.embedding = nn.Embedding(ntoken,
                                d_model)
        self.pos_encoder = PositionalEncoding(d_model,
                                              dropout)
        encoder_layers = TransformerEncoderLayer(d_model,
                                                 nhead,
                                                 d_hid,
                                                 dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers,
                                                      nlayers)
        self.dense = nn.Linear(d_model,3)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.dense.bias.data.zero_()
        self.dense.weight.data.uniform_(-initrange, initrange)

    #データの流れ
    def forward(self, ids, mask):

        # Transformerによるテキストの3値分類           
        x = self.embedding(ids) * math.sqrt(self.d_model) # [batch_size, seq_len, d_model]
        x = self.pos_encoder(x) # [batch_size, seq_len, d_model]
        x = self.transformer_encoder(src=x, src_key_padding_mask=mask) # [batch_size, seq_len, d_model]
        #x = self.transformer_encoder(x) # [batch_size, seq_len, d_model]
        x = x.mean(dim=1) # [batch_size, d_model]
        x = self.dense(x) # [batch_size, 3]

        return x

In [17]:
# Transformer-LSTMモデルの概要
class LSTM(nn.Module):

    def __init__(self,
                 lstm_input_dim: int, 
                 lstm_hidden_dim: int):

        super().__init__()
        self.input_dim = lstm_input_dim
        self.hidden_dim = lstm_hidden_dim
        self.lstm = nn.LSTM(input_size=lstm_input_dim, 
                            hidden_size=lstm_hidden_dim,
                            num_layers=1,
                            batch_first=True)
        self.dense = nn.Linear(lstm_hidden_dim,3)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.dense.bias.data.zero_()
        self.dense.weight.data.uniform_(-initrange, initrange)

    #データの流れ
    def forward(self, src):
            
        # LSTMによるテキスト＋価格の３値分類
        _, x = self.lstm(src)
        print(12, x.size())
        x = self.dense(x[0].view(inlist.size(0), -1))

        return x

In [18]:
# PositionalEncodingの概要
class PositionalEncoding(nn.Module):

    def __init__(self,
                 d_model: int,
                 dropout: float = 0.1,
                 max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        '''
        Args:
            x: Tensor, shape [batch_size, seq_len, embedding_dim]
        '''
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [19]:
# paramator for training & evaluation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
softmax = nn.Softmax(dim=1)
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
torch.manual_seed(0)

In [20]:
# Transformerによるテキストの3値分類
tbatch_size_list       = {'1d':256,  '12h':256,  '4h':128,  '1h':128,  '30m':64,  '15m':64,  '5m':32}

def text_classifer(t_start,t_end):
    tbatch_size = tbatch_size_list[timespan]
    train_tdf   = read_pickle_quickly(f'../../external_drive/pickle/{timespan}/train_tdf.pickle')
    #prob_section = []
    idx_counter_list=[]
    
    # out : list of tesnsor[neg,neu,pos]
    for j in range(t_start,t_end):     
        tlist = CreateDataset1(train_tdf['ids'][j], train_tdf['mask'][j])
        tbatches = iter(DataLoader(tlist, batch_size=128, shuffle=True))
        #p_tbatch = []
        idx_counter = -1
        # out : list of tensor[batch_size,3]
        for idx,batch in enumerate(tbatches):
            idx_counter += 1
            ids  =  batch['ids'].to(device)  # [batch_size, seq_len]
            mask =  torch.t(batch['mask']).to(device) # [seq_len, batch_size]
            text_class = model1(ids, mask)  # [batch_size, 3]
            prob = softmax(text_class) # [batch_size, 3]
            #p_tbatch.append(prob)
            
            write_pickle_quickly(f'../../external_drive/pickle/temp/{j}_{idx}.pickle',prob.to('cpu'))
            del ids, mask, text_class, prob
            torch.cuda.empty_cache()
            gc.collect()
        
        idx_counter_list.append(idx_counter)
#        x = torch.cat(p_tbatch, dim=0) # [batch_size*num_tbatches, 3]
#        x = x.sum(dim=0) # [3]
#        prob_section.append(x)

#    return prob_section
    return idx_counter_list

In [21]:
#train_plist_b[j] のtensorと　prob_sectionのtensorを結合
def create_tensor(t_start,t_end,idx_counter_list):
    prob_section = []
    for j in range(t_start,t_end):
        p_tbatch = []
        for idx in range(0, idx_counter_list[j-t_start]):
            prob = read_pickle_quickly(f'../../external_drive/pickle/temp/{j}_{idx}.pickle')
            p_tbatch.append(prob)
            os.remove(f'../../external_drive/pickle/temp/{j}_{idx}.pickle')
        x = torch.cat(p_tbatch, dim=0) # [batch_size*num_tbatches, 3]
        print(f'1.{x.size()=}')
        x = x.sum(dim=0) # [3]
        print(f'2.{x.size()=}')
        prob_section.append(x)
    
    train_plist = read_pickle_quickly(f'../../external_drive/pickle/{timespan}/train_plist_{n}.pickle')
    src    = []
    target = []
    for j in range(t_start,t_end):
        if train_plist[j]['section'] != -1:
            x = torch.cat(prob_section[j-t_start: j-t_start+n], dim=-1) # [3n]
            print(f'3.{x.size()=}')
            x = torch.cat((x,train_plist[j]['src']), dim=-1).unsqueeze(0) # [1, 5n]
            print(f'4.{x.size()=}')
            src.append(x)
            target.append(train_plist[j]['target'].unsqueeze(0))
    src    = torch.cat(src, dim=0) # [batch_size, 5n]
    print(f'5.{src.size()=}')
    target = torch.cat(target, dim=-1) # [batch_size]
    print(f'6.{target.size()=}')

    return src,target

In [22]:
# training
def train(model1,model2, timespan, n):
    model1.train()
    model2.train()
    
    log_interval = math.ceil(train_num_batches/30)*10
    batch_counter = 0
    train_loss = 0
    train_correct = 0
    train_count = 0
    
    batch_start_time = time.time()
    
    for i in range(0, train_num_batches):
        if i*batch_size-n+1 >= 0:
            
            if i != (train_num_batches-1):
                t_start = i*batch_size-n+1
                t_end   = i*batch_size+batch_size
            else:
                t_start = i*batch_size-n+1
                t_end   = train_num_batches
            
           # prob_section = text_classifer(t_start, t_end)
            idx_counter_list = text_classifer(t_start, t_end)
            print(idx_counter_list)
            src, target = create_tensor(t_start,t_end,idx_counter_list)
            #src, target = create_tensor(prob_section)
            predictions  = model2(src.to(device))
            print('end model2')
            prob = softmax(predictions)
            targets = target.to(device)
            loss = criterion(predictions, targets)

            correct = prob.argmax(axis=1) == targets
            acc = correct.sum().item() / correct.size(0)

            train_correct += correct.sum().item()
            train_count += correct.size(0)
            train_loss += loss.item()

            optimizer1.zero_grad()
            optimizer2.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer1.step()
            optimizer2.step()

        batch_counter += 1

        if (batch_counter % log_interval == 0 or batch_counter == train_num_batches) and (flag == 1):
            lr = scheduler2.get_last_lr()[0]
            s_per_batch = (time.time() - batch_start_time) / log_interval
            cur_loss = train_loss / log_interval
            cur_acc = train_correct / train_count
            print(f'| epoch {epoch:3d} | {batch_counter:5d}/{train_num_batches:5d} batches | '
                  f'lr {lr:1.5f} | s/batch {s_per_batch:5.2f} | '
                  f'loss {cur_loss:5.3f} | accuracy {cur_acc:8.3f}')
            total_loss = 0
            batch_start_time = time.time()

In [23]:
# evaluation (val, test)
def evaluate(model1, model2, timespan, n):
    model1.eval()
    model2.eval()
    test_tdf   = read_pickle_quickly(f'../../external_drive/pickle/{timespan}/test_tdf.pickle')
    test_plist = read_pickle_quickly(f'../../external_drive/pickle/{timespan}/test_plist_{n}.pickle')    
    
    eval_loss = 0
    eval_correct = 0
    eval_count = 0

    with torch.no_grad():
        for i in range(0, test_num_batches):
            if i != (train_num_batches-1):
                t_start = i*batch_size-n+1
                t_end   = i*batch_size+batch_size
            else:
                t_start = i*batch_size-n+1
                t_end   = train_num_batches
        
            prob_section = text_classifer(t_start, t_end)
            src, target = create_tensor(prob_section)
            predictions  = model2(src.to(device))
            prob = softmax(predictions)
            targets = target.to(device)
            loss = criterion(predictions, targets)

            correct = prob.argmax(axis=1) == targets
            eval_acc = correct.sum().item() / correct.size(0)

            eval_correct += correct.sum().item()
            eval_count += correct.size(0)
            eval_loss += loss.item()
        
    print(f'| loss {eval_loss:5.3f}| accuracy {eval_acc:8.3f} ')
     
    return eval_loss, eval_acc

In [24]:
# main
nlist = [1,2,3,4,5,6,7,8,9,10]
tlist = ['1d','12h','4h','1h','30m','15m','5m']
batch_size_list        = {'1d':4,  '12h':8,  '4h':16,  '1h':32,  '30m':64,  '15m':128,  '5m':256}
train_num_section_list = {'1d':181,'12h':363,'4h':1090,'1h':4361,'30m':8722,'15m':17444,'5m':52333}
test_num_section_list  = {'1d':31, '12h':61, '4h':182, '1h':727, '30m':1454,'15m':2908, '5m':8723}
train_num_batches_list = {'1d':46, '12h':46, '4h':69,  '1h':137, '30m':137, '15m':69,   '5m':103}
test_num_batches_list  = {'1d':8,  '12h':8,  '4h':12,  '1h':23,  '30m':23,  '15m':12,   '5m':18}

aculist = {}

nlist = [5]
tlist = ['1h']

for timespan, n in itertools.product(tlist, nlist):

    print('-'*50 + f'{timespan=} / {n=}' + '-'*50)
    train_num_section = train_num_section_list[timespan]
    test_num_section  = test_num_section_list[timespan]
    train_num_batches = train_num_batches_list[timespan]
    test_num_batches  = test_num_batches_list[timespan]
    batch_size=batch_size_list[timespan]

    lr = 1e-3
    model1 = Transformer(ntokens, d_model, nhead, d_hid, nlayers, dropout).to(device)
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=lr)
    scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1, 1.0, gamma=0.95)
    model2 = LSTM(lstm_input_dim, lstm_hidden_dim).to(device)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=lr)
    scheduler2 = torch.optim.lr_scheduler.StepLR(optimizer2, 1.0, gamma=0.95)
    best_val_loss = float('inf')
    epochs = 1
    best_model = None

    dt_start = datetime.datetime.now()
    print(datetime.datetime.now())
    print('*'*45 + 'training start' + '*'*45)

    # training & test roop
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()

        train(model1, model2, timespan, n)
        val_loss, val_acc = evaluate(model1, model2, timespan, n)

        print('-' * 95)
        print(f'| end of epoch {epoch:3d} | time: {time.time()-epoch_start_time:5.2f}s | '
              f'val loss：{val_loss:5.3f} | val accuracy：{val_acc:8.3f}')
        print('-' * 95)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model1 = copy.deepcopy(model1)
            best_model2 = copy.deepcopy(model2)

        scheduler1.step()
        scheduler2.step()

        del epoch_start_time, val_loss, val_acc
        gc.collect()

    dt_end = datetime.datetime.now()
    elapsed = dt_end - dt_start
    print(datetime.datetime.now())    
    print('*'*30 + f'Finish! training time：{elapsed:8.2f}s' + '*'*30)

    # test
    test_loss, test_acc = evaluate(best_model1, best_model2, timespan, n)
    print('=' * 89)
    print(f'| End of training | test loss：{test_loss:5.3f} | '
          f'test accuracy：{test_acc:8.3f}')
    print('=' * 89)

--------------------------------------------------timespan='1h' / n=5--------------------------------------------------
2021-11-08 08:41:04.949844
*********************************************training start*********************************************
[7, 8, 10, 10, 13, 10, 9, 9, 40, 39, 36, 35, 67, 49, 40, 39, 33, 26, 22, 18, 16, 16, 14, 13, 12, 28, 20, 26, 23, 20, 18, 19, 23, 24, 24, 25]
1.x.size()=torch.Size([896, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1024, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1280, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1280, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1664, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1280, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1152, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([1152, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([5120, 3])
2.x.size()=torch.Size([3])
1.x.size()=torch.Size([4992, 3])
2.x.size()=torch.Size([3])
1.x.size

RuntimeError: Sizes of tensors must match except in dimension 0. Got 25 and 22 in dimension 1 (The offending index is 32)

In [ ]:
#device = 'cpu'
#device = torch.device("cuda:1")
print(device)
#torch.cuda.is_initialized()
#torch.cuda.ipc_collect()
#torch.cuda.empty_cache()
#gc.collect()
#print(f'{sys.getsizeof(l)=}')

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device0 = torch.device("cuda:0")
device1 = torch.device("cuda:1")
print(torch.cuda.is_available())
print(device)

True
cuda


In [ ]:
timespan = '1h'    
print('-'*50 + f'{timespan=}' + '-'*50)
train_tdf, test_tdf = data_process1(timespan)
print(f'{sys.getsizeof(train_tdf)=}')
print(f'{sys.getsizeof(test_tdf)=}')
train_tdf.to_csv(f'../../external_drive/pandas/{timespan}/train_tdf.csv', index=False)
test_tdf.to_csv(f'../../external_drive/pandas/{timespan}/test_tdf.csv', index=False)

In [19]:
a = torch.randn(3,3).to(device)
print(a)

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193]], device='cuda:0')


In [20]:
print(a.to('cpu'))

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193]])
